# Building RAG system with langchain and chromaDB

## Introduction

RAG -

    - Langchain: for developing application
    - ChromaDB: open source vector db
    - openAi: for embedding and language

In [8]:
from dotenv import load_dotenv
load_dotenv()

False

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings

from langchain_community.vectorstores import Chroma



In [7]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    "files",
    glob='*.txt',
    loader_cls=TextLoader,
    loader_kwargs={'encoding':'utf-8'}
)

documents = loader.load()


In [8]:
# Document Splitting

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

print(f"chunks with len {len(chunks)}")


chunks with len 6


In [10]:
# Embeddings

from langchain_huggingface import HuggingFaceEmbeddings

## Initialise a simple Embedding model (no key needed)

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

embeddings.embed_query(documents)

AttributeError: 'list' object has no attribute 'replace'

### Initialize ChromaDB vector store and store the chunk

In [25]:
# persistant directory
path="./chroma_db"

# init chroma db with hugging face embedding
vector_store=Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=path,
    collection_name="rag_collection"
)



### Test similarity search

In [14]:
query = "What is Artificial Intelligence?"

similar_docs=vector_store.similarity_search(query)
similar_docs

[Document(metadata={'source': 'files/ai.txt'}, page_content='8. Artificial Intelligence'),
 Document(metadata={'source': 'files/ai.txt'}, page_content='Modern AI isn’t intelligent in the human sense. It’s a pattern amplifier trained on oceans of data. What makes it powerful isn’t creativity—it’s speed and scale. It can generate, compare, and'),
 Document(metadata={'source': 'files/ai.txt'}, page_content='compare, and optimize patterns faster than humans ever will, but it still lacks true understanding.'),
 Document(metadata={'source': 'files/climatchange.txt'}, page_content='Solutions exist, but they’re expensive upfront and pay off slowly, so decision-makers stall.')]

### Similarity search 

Low score more similar

In [16]:
query = "What is Artificial Intelligence?"
similar_docs = vector_store.similarity_search_with_score(query)
similar_docs

[(Document(metadata={'source': 'files/ai.txt'}, page_content='8. Artificial Intelligence'),
  0.6806873679161072),
 (Document(metadata={'source': 'files/ai.txt'}, page_content='Modern AI isn’t intelligent in the human sense. It’s a pattern amplifier trained on oceans of data. What makes it powerful isn’t creativity—it’s speed and scale. It can generate, compare, and'),
  0.739311695098877),
 (Document(metadata={'source': 'files/ai.txt'}, page_content='compare, and optimize patterns faster than humans ever will, but it still lacks true understanding.'),
  1.2848765850067139),
 (Document(metadata={'source': 'files/climatchange.txt'}, page_content='Solutions exist, but they’re expensive upfront and pay off slowly, so decision-makers stall.'),
  1.7676345109939575)]

### Initialize LLM, RAG chain,prompt template , Query the RAG system

In [24]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv() 
# openai_key = os.environ.get('OPENAI_KEY')
llm=ChatOpenAI(
    model= "gpt-3.5-turbo",
    temperature= 0)

respose=llm.invoke("What is llm?")
respose

AIMessage(content='LLM stands for Master of Laws, which is a postgraduate academic degree typically pursued by individuals who have already completed a law degree and wish to further specialize in a specific area of law or gain expertise in a particular legal field. The LLM degree is recognized internationally and can lead to career opportunities in various legal sectors.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 12, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CkZk592xWlhyFJC9v8mUUkKEH51T7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--6731ce3f-9159-4c7d-90f2-17c9453de9e7-0', usage_metadata=

### Mordern RAG Chain

In [26]:
# Convert vector store to retriever
retriever=vector_store.as_retriever(
    search_kwarg={"k":3} ## retrive top 3 relevent chunks
)
retriever


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x1663ca3f0>, search_kwargs={})

In [27]:
from langchain_core.prompts import ChatPromptTemplate
# Prompt template

system_prompt="""You are an assistant for question-answering tasks.
 Use following pieces of rerieved context to ans the question. 
 
 Context: {context}"""

prompt= ChatPromptTemplate.from_messages([
    ("system",system_prompt),
    ("human","{input}")
])

In [32]:
# Create document chain - ie combine relevent document after extracting from vector store
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are an assistant for question-answering tasks.\n Use following pieces of rerieved context to ans the question. \n\n Context: {context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1666da210>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1666da5d0>, root_client=<openai.OpenAI object at 0x1666d96d0>, root_a

This chain

- takes retrieved documents
- "stuffs" them into the prompts {{context}} placeholder
- sends the complete prompt to the llm
- returns the llm's response

In [33]:
# final RAG chain

rag_chain=create_retrieval_chain(retriever,document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x1663ca3f0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are an assistant for question-answering tasks.\n Use following pieces of rerieved context to ans the question. \n\n Context: {context}

In [36]:
rag_chain.invoke({"input":"What is real challange in climate change"})

{'input': 'What is real challange in climate change',
 'context': [Document(metadata={'source': 'files/climatchange.txt'}, page_content="Climate change isn’t a debate; it's a measurement. CO₂ rises, temperatures follow, and feedback loops accelerate the problem. The real challenge isn’t science—it’s politics and economics. Solutions"),
  Document(metadata={'source': 'files/climatchange.txt'}, page_content="Climate change isn’t a debate; it's a measurement. CO₂ rises, temperatures follow, and feedback loops accelerate the problem. The real challenge isn’t science—it’s politics and economics. Solutions"),
  Document(metadata={'source': 'files/climatchange.txt'}, page_content='9. Climate Change'),
  Document(metadata={'source': 'files/climatchange.txt'}, page_content='9. Climate Change')],
 'answer': 'The real challenge in climate change is not the science behind it, but rather the politics and economics involved in finding and implementing solutions.'}